Write programs to solve a set of uniform random 3-SAT problems for different combinations of m and n, and compare their performance.  Try the Hill-Climbing, Beam-Search with beam widths 3 and 4, Variable-Neighborhood-Descent with 3 neighborhood functions.  Use two different heuristic functions and compare them with respect to penetrance.



In [ ]:
import random

def generate_3sat_problem(m, n):
    return [tuple(random.choice([random.randint(1, n), -random.randint(1, n)]) for _ in range(3)) for _ in range(m)]

def evaluate_assignment(assignment, clauses):
    return sum(1 for clause in clauses if any((lit > 0 and assignment[abs(lit) - 1]) or (lit < 0 and not assignment[abs(lit) - 1]) for lit in clause))

def hill_climbing(clauses, n, max_iterations=1000):
    best_assignment = [random.choice([True, False]) for _ in range(n)]
    best_score = evaluate_assignment(best_assignment, clauses)

    for _ in range(max_iterations):
        flip_index = random.randint(0, n - 1)
        neighbor = best_assignment[:]
        neighbor[flip_index] = not neighbor[flip_index]
        neighbor_score = evaluate_assignment(neighbor, clauses)

        if neighbor_score > best_score:
            best_assignment, best_score = neighbor, neighbor_score

        if best_score == len(clauses):
            return True

    return best_score == len(clauses)

def beam_search(clauses, n, beam_width, max_iterations=1000):
    beam = [[random.choice([True, False]) for _ in range(n)] for _ in range(beam_width)]
    best_score = max(evaluate_assignment(b, clauses) for b in beam)

    for _ in range(max_iterations):
        neighbors = []
        for assignment in beam:
            for i in range(n):
                neighbor = assignment[:]
                neighbor[i] = not neighbor[i]
                neighbors.append(neighbor)

        scores_and_neighbors = sorted([(evaluate_assignment(neighbor, clauses), neighbor) for neighbor in neighbors], reverse=True)[:beam_width]
        beam = [neighbor for _, neighbor in scores_and_neighbors]

        if scores_and_neighbors[0][0] == len(clauses):
            return True

    return best_score == len(clauses)

def variable_neighborhood_descent(clauses, n, max_iterations=1000):
    def flip_k_bits(assignment, k):
        indices = random.sample(range(n), k)
        return [not assignment[i] if i in indices else assignment[i] for i in range(n)]

    best_assignment = [random.choice([True, False]) for _ in range(n)]
    best_score = evaluate_assignment(best_assignment, clauses)

    for _ in range(max_iterations):
        for k in [1, 2, 3]:
            neighbor = flip_k_bits(best_assignment, k)
            neighbor_score = evaluate_assignment(neighbor, clauses)

            if neighbor_score > best_score:
                best_assignment, best_score = neighbor, neighbor_score
                break

        if best_score == len(clauses):
            return True

    return best_score == len(clauses)

def test_algorithms(m_values, n_values, runs=10):
    results = []

    for m, n in zip(m_values, n_values):
        penetrance = {
            'Hill Climbing': 0,
            'Beam Search (width=3)': 0,
            'Beam Search (width=4)': 0,
            'VND': 0,
        }

        for _ in range(runs):
            clauses = generate_3sat_problem(m, n)

            if hill_climbing(clauses, n):
                penetrance['Hill Climbing'] += 1

            if beam_search(clauses, n, beam_width=3):
                penetrance['Beam Search (width=3)'] += 1

            if beam_search(clauses, n, beam_width=4):
                penetrance['Beam Search (width=4)'] += 1

            if variable_neighborhood_descent(clauses, n):
                penetrance['VND'] += 1

        for key in penetrance:
            penetrance[key] = (penetrance[key] / runs) * 100

        results.append((m, n, penetrance))

    for m, n, penetrance in results:
        print(f"Testing with m-{m}, n-{n}:")
        for key, value in penetrance.items():
            print(f"{key} penetrance: {value:.2f}%")

if __name__ == "__main__":
    m_values = input("Enter values for m (comma-separated): ").split(',')
    n_values = input("Enter values for n (comma-separated): ").split(',')

    # Convert the inputs to integers
    m_values = list(map(int, m_values))
    n_values = list(map(int, n_values))

    # Ensure both lists have the same length
    if len(m_values) != len(n_values):
        print("Error: The number of m values must match the number of n values.")
    else:
        test_algorithms(m_values, n_values, runs=5)

Enter values for m (comma-separated): 20,20,20,40,40
Enter values for n (comma-separated): 10,20,30,10,20
Testing with m-20, n-10:
Hill Climbing penetrance: 80.00%
Beam Search (width=3) penetrance: 100.00%
Beam Search (width=4) penetrance: 100.00%
VND penetrance: 100.00%
Testing with m-20, n-20:
Hill Climbing penetrance: 100.00%
Beam Search (width=3) penetrance: 100.00%
Beam Search (width=4) penetrance: 100.00%
VND penetrance: 100.00%
Testing with m-20, n-30:
Hill Climbing penetrance: 100.00%
Beam Search (width=3) penetrance: 100.00%
Beam Search (width=4) penetrance: 100.00%
VND penetrance: 100.00%
Testing with m-40, n-10:
Hill Climbing penetrance: 20.00%
Beam Search (width=3) penetrance: 40.00%
Beam Search (width=4) penetrance: 60.00%
VND penetrance: 40.00%
Testing with m-40, n-20:
Hill Climbing penetrance: 80.00%
Beam Search (width=3) penetrance: 80.00%
Beam Search (width=4) penetrance: 80.00%
VND penetrance: 100.00%
